In [1]:
from download import *

import pandas as pd



In [2]:
# read real db export and training data

training_data = pd.read_csv("posts.csv")
e6_dump_data = pd.read_csv("posts-2023-04-08.csv")

In [3]:
# inner join
subset_e6_dump_data = e6_dump_data[e6_dump_data["md5"].isin(training_data["md5"].tolist())]

In [4]:
# static e6 url
# file_ext
# md5
file_ext = "jpg"
md5 = "1234567890"
f'https://static1.e621.net/data/{md5[0:2]}/{md5[2:4]}/{md5}.{file_ext}'

'https://static1.e621.net/data/12/34/1234567890.jpg'

In [5]:
subset_e6_dump_data.columns

Index(['id', 'uploader_id', 'created_at', 'md5', 'source', 'rating',
       'image_width', 'image_height', 'tag_string', 'locked_tags', 'fav_count',
       'file_ext', 'parent_id', 'change_seq', 'approver_id', 'file_size',
       'comment_count', 'description', 'duration', 'updated_at', 'is_deleted',
       'is_pending', 'is_flagged', 'score', 'up_score', 'down_score',
       'is_rating_locked', 'is_status_locked', 'is_note_locked'],
      dtype='object')

In [6]:
# test retrieving image 
import os


dataset = subset_e6_dump_data.sample(10, random_state = 1)

def download(dataset:pd.DataFrame, save_path:str):
    
    
    for index, sample in dataset.iterrows():
        
        print(index)
        
        attempt = 0
        while attempt < 10:
            try:
                # generate url
                file_ext = sample["file_ext"]
                md5 = sample["md5"]
                url = f'https://static1.e621.net/data/{md5[0:2]}/{md5[2:4]}/{md5}.{file_ext}'

                user_agent_message = (
                    f"heya! this is lodestone from furry diffusion server, "
                    f"i need to rebuild my dataset for training, "
                    f"please let me know if this bot is pulling data to fast "
                    f"PS: OOPS sorry! too fast"
                )

                # download image as PIL object
                image = stream_image(url, user_agent=user_agent_message, threshold_size = 0)

                image = rescale_image(image, 1024)

                save_webp_without_alpha(
                    image, 
                    os.path.join(save_path, f"{md5}.webp"), 
                    quality=70
                )
                break
            except:
                print(f"failed downloading {index} ... retrying {attempt}/10")
                attempt += 1
        else:
            
            print(f"download attempt exceeded skipping {index}")
            continue 

# download(dataset)

In [7]:
list_image = os.listdir("/home/user/project-fur/e6_dump/1024")
list_image

['1f74656c9ecbbd43e51572c869dbea98.webp',
 '5a9f3b87c4da9dd7b294cdf84825276b.webp',
 '4f0b6d73d30585326cf943dffc6eedff.webp',
 '11fa2426d5c8d1f99aba6edac515e0f9.webp',
 'ee3f6385f827ced11a20cd7108afbf9d.webp',
 'd764c0e6d334a135d322a2997067c0a6.webp',
 'e8d58b94df985ffe1469536e6f541a81.webp',
 '501035d0e13cf05344bf0f0c97513f46.webp',
 '19ba011894d670120490eebe16d07768.webp',
 '360f3bda1d593fff6fefa9a7783d6111.webp',
 'bc820ca9e6fd9d29c649b89f4fc4fae7.webp',
 '93cde3701995725f6814f34508db1de4.webp',
 '44d474343e70fc6c23a61dc6fe8867e1.webp',
 'e9fe1052678733aa08628cf6a408f833.webp',
 '9f290121c1dad724d8a2081c8e51ff58.webp',
 'ccfeb7e07e5e7fa071b9e3b2d08e3ce1.webp',
 '88b8732d0bfe06bcaeedda4593661e5f.webp',
 '4b80188f6203b930a49b27986efecc69.webp',
 '96bfda22f0a1c37828c12fe6279068e5.webp',
 '0894326415e614d2897709bef9ca79e1.webp',
 'aa7f5fa711846a660591f5771f6f9f8b.webp',
 '38aa8767141fc47ec15044997ab61ac8.webp',
 '28c4c40b748e73a3bdaad12f41d56460.webp',
 'bbc2f78afd88b56c6cdc7ac65d170a3d

In [8]:
list_image_pd = pd.DataFrame(list_image)
list_image_pd = list_image_pd.iloc[:,0].str.replace(".webp","")

/tmp/ipykernel_1151588/2261591160.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  list_image_pd = list_image_pd.iloc[:,0].str.replace(".webp","")


In [9]:
list_image_pd.tolist()

['1f74656c9ecbbd43e51572c869dbea98',
 '5a9f3b87c4da9dd7b294cdf84825276b',
 '4f0b6d73d30585326cf943dffc6eedff',
 '11fa2426d5c8d1f99aba6edac515e0f9',
 'ee3f6385f827ced11a20cd7108afbf9d',
 'd764c0e6d334a135d322a2997067c0a6',
 'e8d58b94df985ffe1469536e6f541a81',
 '501035d0e13cf05344bf0f0c97513f46',
 '19ba011894d670120490eebe16d07768',
 '360f3bda1d593fff6fefa9a7783d6111',
 'bc820ca9e6fd9d29c649b89f4fc4fae7',
 '93cde3701995725f6814f34508db1de4',
 '44d474343e70fc6c23a61dc6fe8867e1',
 'e9fe1052678733aa08628cf6a408f833',
 '9f290121c1dad724d8a2081c8e51ff58',
 'ccfeb7e07e5e7fa071b9e3b2d08e3ce1',
 '88b8732d0bfe06bcaeedda4593661e5f',
 '4b80188f6203b930a49b27986efecc69',
 '96bfda22f0a1c37828c12fe6279068e5',
 '0894326415e614d2897709bef9ca79e1',
 'aa7f5fa711846a660591f5771f6f9f8b',
 '38aa8767141fc47ec15044997ab61ac8',
 '28c4c40b748e73a3bdaad12f41d56460',
 'bbc2f78afd88b56c6cdc7ac65d170a3d',
 '480506cf7c9139ff154d8c53a432b115',
 '010476451c94b62370f065eb339b29a8',
 '29ae165e07721b0d62592f46a054f5f3',
 

In [10]:
subset_e6_dump_data = subset_e6_dump_data[~subset_e6_dump_data["md5"].isin(list_image_pd.tolist())]

In [11]:
subset_e6_dump_data

,id,uploader_id,created_at,md5,source,rating,image_width,image_height,tag_string,locked_tags,...,updated_at,is_deleted,is_pending,is_flagged,score,up_score,down_score,is_rating_locked,is_status_locked,is_note_locked
37549,38373,3844,2009-08-01 05:32:32.574629,3c85f72359d2c08c88beaa4168fa5929,https://areaoutlaw.h.fc2.com/cg_0049.htm,e,800,632,aged_up anthro bdsm big_bad_wolf black_nose bl...,NaN,...,2023-04-05 23:16:21.130625,f,f,f,178,190,-11,t,f,f
37616,38443,3835,2009-08-02 04:35:54.437426,0c78057455a58f2f44481e8b21bef934,https://furaffinity.net/full/2596374/\nhttps:/...,e,1280,681,anthro anthro_on_feral anus avian beak bestial...,NaN,...,2023-04-02 03:19:36.086599,f,f,f,139,139,-2,t,f,f
37682,38509,3938,2009-08-03 09:44:03.570259,2395ca079aa7e522424610f7a2f2be1d,https://www.furaffinity.net/view/1444944/,s,722,709,2008 5_fingers alcohol anthro beverage blotch ...,NaN,...,2023-03-27 06:04:41.643586,f,f,f,162,162,0,f,f,f
37691,38518,2594,2009-08-03 11:26:15.766333,b6bc298c1c6f3a0749373d7a3892a913,https://furrynetwork.com/artwork/217826\nhttps...,e,900,720,5:4 <3 anthro arms_bent backsack balls bent_ar...,NaN,...,2023-04-03 01:56:53.205513,f,f,f,93,95,-3,t,f,f
37700,38527,5616,2009-08-03 16:45:18.291276,b9f308b5a8f77f8d8e4ad25a2bc6a653,NaN,e,1028,1515,1997 anthro anus bodily_fluids breasts clitori...,NaN,...,2023-03-27 21:11:22.84244,f,f,f,104,104,-1,t,f,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3867367,3875523,45665,2023-02-16 01:56:33.177574,d59641ab8069d2aa9805af91453b4461,https://pbs.twimg.com/media/E3AotXVVcAUxk9p?fo...,e,1200,883,2021 5ushiroll <3 ahegao all_fours anal anal_p...,NaN,...,2023-04-08 03:00:35.825786,f,f,f,360,361,-1,f,f,f
3867463,3875619,242844,2023-02-16 02:48:48.380387,ebb33374ee52d868d26f4dcaa5130e5a,https://twitter.com/Elicitie/status/1626004923...,e,3784,5826,2023 <3 absurd_res all_fours ambiguous_gender ...,NaN,...,2023-04-08 01:08:55.356206,f,f,f,1204,1220,-16,f,f,f
3867543,3875699,153320,2023-02-16 03:57:08.350325,3317fde606ca8a53e6242161e614fa08,https://twitter.com/elz_zombie/status/16259665...,e,2400,2000,2023 6:5 <3 ahegao animal_genitalia animal_pen...,NaN,...,2023-04-08 02:21:19.493473,f,f,f,1017,1030,-13,f,f,f
3867607,3875763,688534,2023-02-16 05:00:21.028637,603529ff72abc6744d34266aff0520de,NaN,e,2786,3754,<3 absurd_res animal_genitalia anthro anus arm...,conditional_dnp,...,2023-04-08 05:35:57.322205,f,f,f,886,908,-22,t,f,f


In [12]:
from threading import Thread

def multithread_download(df:pd.DataFrame, save_path:str, number_of_workers:int = 10) -> None:

    # function tp be executed as threads
    split_df = split_dataframe(df, number_of_workers)

    threads = []
    for df in split_df:
        thread_instance = Thread(target=download, args=[df, save_path])
        threads.append(thread_instance)

    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()
        


In [ ]:
import multiprocessing

number_of_workers = 10
split_df = split_dataframe(subset_e6_dump_data, number_of_workers)
save_path = "/home/user/project-fur/e6_dump/1024"

args = [(df,) + (save_path,) + (10,) for df in split_df]

with multiprocessing.Pool(processes=number_of_workers) as pool:
    results = pool.starmap(multithread_download, args)
    print(results)


37549257141
392518
593902
688239774139

509509
947759855935


1031753
1119108
1198777126112113249101382296



156036316100411502967
16590821442486



1709158
1753926179583518359581031780

1917442

1959071

1877071204025120009482076260



509512
37616
855954
21144372150104
22174492183834


2248048774141257211

2277368

23289872303145
23843282358013


688272
947786
1502981
392554
1324913
1382302
1261142
24344722458969
2484961241039625078722532563



2557930

258014226034022625702


1119137
1198778
1659085
1442514
593908
1917445
1031784
1560369
26706362648902

37682
276022627150602692500
27368722804521
2781775
2848848

2826525


509516
257212
2384329
1610053
20009562076320

2277371
1709161
1795840
1959072
28718082893650

295818929151252936793


300112130226582979104


30458893066829

392562
2303148
774142
2040256
1382306
1835975
2410398
23289972484967

30885446882872248053
3154720
31752861877074

3109887

313163131975803219367
32659283241170

3289527

2625708


2434479
2557932
2114457
250

1836024
3022708
509883
2625742
2532603
1503136
Error opening image https://static1.e621.net/data/6c/f2/6cf2277efdda27c6daac54f385229495.jpg: cannot identify image file <_io.BytesIO object at 0x7fe8dca79d60>
failed downloading 2979123 ... retrying 5/10
3219390
3699898
3314012
3154753
1442660
3131665
28718591325013

3482209
2485011
1959103
2893677
9479833507427

3757654
26489623671057
856191

1795927
3926612303177

7743512329049

1709233
2603438
3001148
2670683
3641855
3109935
1198937
2848909
3197647
3790234
3611920
2692525
38346
1917504
3532131
2459066
688451
Error opening image https://static1.e621.net/data/6c/f2/6cf2277efdda27c6daac54f385229495.jpg: cannot identify image file <_io.BytesIO object at 0x7fe8dca81e50>
failed downloading 2979123 ... retrying 6/10
3456352
3265965
2915154
37288901659170

1877101
1754083
2507941
3824188
3407248
2781821
2076416
2277408
3045906
3241206
1119347
2358054
257537
2736909
2826601
2410474
2760291
594302
1560463
3383990
3175317
2248104
2434553
2580190


856269
2781870
1325074
1836237
3433023
2277450
3314066
2217553
3790273
2384402
3699942
3131688
3241222
1917522
1119460
1560566
1442860
2183957
1031981
2603480
1795971
3757712
510042
Error opening image https://static1.e621.net/data/2d/cd/2dcdc89b2fd2f5c26c6709702551a9c2.jpg: cannot identify image file <_io.BytesIO object at 0x7fe8dc211540>
failed downloading 2114559 ... retrying 1/10
2459168
1709406
2076454
28936933088644

1877120
1503246
2150212
3641903
2848961
3558947
1959200
3456366
3359017
2715174

2001058688538
2580221
1754124
3109998
2760330
2248159
2915190
3384028
2507977
3532169
29791392329071

2558059
948275
2434598
2625787
1610220
1199068
3585583
2303233
3197693
2826622
1261393
Error opening image https://static1.e621.net/data/2d/cd/2dcdc89b2fd2f5c26c6709702551a9c2.jpg: cannot identify image file <_io.BytesIO object at 0x7fe78771cbd0>
failed downloading 2114559 ... retrying 2/10
2871886
38944
2040376
3001193
2736940
594432
3022748
3507440
257621
3433028
2485090
3265996
317535

2277477
2076487
3559047
2826666
3456444
2915215
2893715
3824351
39514
2603554
1610276
2508011
2958288
3154793
2434631
2329119
2384457
22175942871907

2692598
1959322
3266017
2248198
3790337
3359068
688668
3482302
1796087
3022761
1836335
3700016
2804615
774744
2040394
2184050
2580260
2649078
2114621
3641926
1754223
3384056
3757759
35856432303268

1032002
2532661
1325537
3045937
1199125
2670770
393074
3289626
1917587
3175393
1560683
2736973
594737
1442939
2358116
948513
2936914
2410512
3612006
1503462
22774821709540

257917
1119560
2001118
1261470
2485122
3314150
2459218
3197758
510198
2849005
39658
27603773532244

1877152
2558123
1836357
3700017
3559060
2958289
1382502
3088682
1610310
2979215
3507481
37290322076492

3335893
2150276
2603555
3824355
2329124
2434648
3433074
856452
1796090
3241268
2781942
3110014
3671166
2508014
3641931
688669
3219483
2692600
3001244
3066964
28266723359069

3482308
1659406
28719192915234

2114624
1959342
2384463
2715211
3154798
3700025
2248200
774749
375776

/home/user/project-fur/fur-env/lib/python3.8/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (102000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


...

36748143485956

3829808
1567653
Error opening image https://static1.e621.net/data/06/37/0637b4f5d6c7bb2e2f3f38d867e1212c.jpg: cannot identify image file <_io.BytesIO object at 0x7fe7876aa270>
failed downloading 3410884 ... retrying 6/10
2387627
2808058
3293176
2785184
1802131
280900
2155926
1842593
703094
2561102
3178656
3339302
788195
2536577
2829795
3048911
2918551
3387485
1965540
3268764
28751281133555

2306932
31129923004150

3733106
2583946
2695800
3535971
411762
2982366
2281158
2718313
3200908
Error opening image https://static1.e621.net/data/06/37/0637b4f5d6c7bb2e2f3f38d867e1212c.jpg: cannot identify image file <_io.BytesIO object at 0x7fe8d4f74ae0>
failed downloading 3410884 ... retrying 7/10
1760353
2007238
1208808
1334347
2960986
2045938
3562858
2362146
2438230
3091742
1715361
2940435
3244329
1882784
3436158
2119998
3157698
2081365
3135039
2896873
2763746
2488532
1451642
2414427
1512919
1044782
101039
1390946
2673913
2852574
3510884
1269328
Error opening image https://static1

Error opening image https://static1.e621.net/data/e5/b5/e5b557f2a7e1239937c99faa2f4abef7.png: cannot identify image file <_io.BytesIO object at 0x7fe8dfbaad10>2387680

1760462
failed downloading 1334362 ... retrying 7/10
3362475
Error opening image https://static1.e621.net/data/96/89/968951d13dd152ad850d60567fb6accd.png: cannot identify image file <_io.BytesIO object at 0x7fe8d4cade50>
failed downloading 1208963 ... retrying 2/103387512

411902
1567661
9606981451740

2829820
2652001
36748501802229

3703961
3113014
1044911
3244350
3070264
3536008
2007308
2718339
2414458
3645836
Error opening image https://static1.e621.net/data/9b/74/9b744a4015ae4c540827ea5e20226e3c.jpg: cannot identify image file <_io.BytesIO object at 0x7fe8c707e810>
failed downloading 2607030 ... retrying 4/10
1512954
609137
3829874
3761381
2808088
16666152561118

1882792
281112
2045946
3222647
3795402
3589371
2511702
2462940
1617710
3410936
Error opening image https://static1.e621.net/data/e5/b5/e5b557f2a7e1239937c99

Error opening image https://static1.e621.net/data/46/17/461732e367545782b68d69c58bd53e93.jpg: cannot identify image file <_io.BytesIO object at 0x7fe8d4cd2400>
failed downloading 3616144 ... retrying 7/10
281163
2740027
2120012
3004193
3510930
3733161
2462954
3091797
1965692
2007322
1567672
2253004
1512985
2362176
1045024
609177
1334424
3200952
1209107
2808109
1760491
2896963
960773
3436236
3049028
2438271
Error opening image https://static1.e621.net/data/46/17/461732e367545782b68d69c58bd53e93.jpg: cannot identify image file <_io.BytesIO object at 0x7fe8d59603b0>
failed downloading 3616144 ... retrying 8/10
379543427638323829914


3222671
3645878
3410981
2306994
2829857
2045987
411977
2584004
2536704
1715434
1666658
524590
3268846
1133816
2718349
3536032
2156006
2940484
3387520
3135107
3113033
3339346
31787272414473
101903

1923334
2629002
2918577
3317188
3589401
Error opening image https://static1.e621.net/data/46/17/461732e367545782b68d69c58bd53e93.jpg: cannot identify image file <_i

2982510
2673964
2333233
36749603795537

2718402
2940510
1802361
3026223
2629025
1567753
2462998
3830076
2188769
2561169
3339386
2852692
3436271
1334573
960983
2438360
2740110
3222727
2918625
1045145
1391184
3645949
2488622
3510970
3733193
281440
3387568
3460163
2253079
2808194
3201031
2652053
524657
18429062222016

3049107
102350
2007340
3704016
3244416
3268936
1760595
2511754
609454
3070318
19658673536063

1923419
2046059
2387755
3004239
1715521
3113069
2829949
2584030
2536752
2362206
703328
1209324
412302
2120117
2785368
3157756
868795
3616217
2307041
1666773
3486063
35894792961053

1617826
2982512
1334592
1882884
1513027
2281258
1802373
2188774
2414493
2081503
2607087
1134014
2940511
3563003
2718403
269588437615043293281


3674966
2897073
2852698
2875208
3091844
3411065
1045168
3795541
3178782
3436275
1452020
961006
2808213
2333235
2156051
3026228
1567770
1269597
102358
2629030
28144413911892438362


2763868
3339389
3135153
3317227
3830080
3049111
2222019
788577
2740112
2673970
2561

26520933436327

2438381
3201063
1452104
1045261
3070351
1802441
2940520
3113112
2763893
103103
2875243
788669
3795583
2918698
3339418
2414506
3645994
3004260
3317285
3049119
1760668
3091862
3486135
2188855
1617895
3222776
3268968
2362274
2897112
524967
2561191
2156072
868907
2982562
2081581
3026306
23071092808282

3460240
2387832
3616288
3178812
2718460
2281305
1965934
35895321513136

2253135
1391249
3704027
3563052
412353
17155951567815

609733
3157782
3387626
3411104
34363661334673

1883033
3244464
1134092
2961063
2222059
2120144
3675033
1209440
3362657
3733238
3830156
2652096
1269651
25117953070353

2607117
1666879
281712
703661
20461293761567

2463064
2584092
3135183
2740156
2629064103217

2536790
2875268
1923505
2940521
24886442785456

2673997
961143
3004262
2333311
1843061
788688
3511055
2763904
2696030
3201065
2438382
3293324
3113113
1045311
2918703
3795585
1452129
1802443
3049120
2830023
2982564
2414532
1617902
23622792718464

3616291
524970
3157783
25611943536132

1760670
2188

In [29]:
# constant variable
save_path = "/home/user/project-fur/e6_dump/1024"

[<Thread(Thread-5, initial)>,
 <Thread(Thread-6, initial)>,
 <Thread(Thread-7, initial)>,
 <Thread(Thread-8, initial)>,
 <Thread(Thread-9, initial)>,
 <Thread(Thread-10, initial)>,
 <Thread(Thread-11, initial)>,
 <Thread(Thread-12, initial)>,
 <Thread(Thread-13, initial)>,
 <Thread(Thread-14, initial)>,
 <Thread(Thread-15, initial)>,
 <Thread(Thread-16, initial)>,
 <Thread(Thread-17, initial)>,
 <Thread(Thread-18, initial)>,
 <Thread(Thread-19, initial)>,
 <Thread(Thread-20, initial)>,
 <Thread(Thread-21, initial)>,
 <Thread(Thread-22, initial)>,
 <Thread(Thread-23, initial)>,
 <Thread(Thread-24, initial)>,
 <Thread(Thread-25, initial)>,
 <Thread(Thread-26, initial)>,
 <Thread(Thread-27, initial)>,
 <Thread(Thread-28, initial)>,
 <Thread(Thread-29, initial)>,
 <Thread(Thread-30, initial)>,
 <Thread(Thread-31, initial)>,
 <Thread(Thread-32, initial)>,
 <Thread(Thread-33, initial)>,
 <Thread(Thread-34, initial)>,
 <Thread(Thread-35, initial)>,
 <Thread(Thread-36, initial)>,
 <Thread(Thre

In [3]:
!mkdir /home/user/project-fur/e6_dump/1024